### DataLoding

In [ ]:
# data["item_id"] = data["item_id"].apply(lambda x: mapping.get(x, x))

In [1]:
from data_cleaning import run_listen_pipeline

## Кодирование данных


кодируем uid и item_id в числовые индексы, чтобы модели и алгоритмы могли с ними работать.

Строковые ID нельзя напрямую подать в матрицу взаимодействий или Embedding-слой, поэтому мы переводим их в диапазон 0..N.
Это нужно для эффективности (память, скорость), совместимости с моделями (Matrix Factorization, Embeddings) и стабильного маппинга между train и test.

In [2]:
def codic_of_index(train_df, test_df):
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    user_id_map = {uid: idx for idx, uid in enumerate(train_df["uid"].unique())}
    item_id_map = {iid: idx for idx, iid in enumerate(train_df["item_id"].unique())}
    
    train_df["uid"] = train_df["uid"].map(user_id_map)
    train_df["item_id"] = train_df["item_id"].map(item_id_map)
    
    test_df["uid"] = test_df["uid"].map(user_id_map)
    test_df["item_id"] = test_df["item_id"].map(item_id_map)

    return train_df, test_df, item_id_map

### Тест трейн сплит 

Сразу разделим данные на тестову и обучающую выборку.

Деление будет по временному признаку. 
Чтобы исключить dataLeakege (когда признаки, рассчитанные на истории,
могут «заглядывать» в будущее), вставим временной зазор (gap)

В оригинальной статье https://www.arxiv.org/pdf/2505.22238 берется соотношение 300, 30мин, 1 день

Лучшие результаты у меня получились при  80, 30мин, 20 день

Чтобы приблизиться к условиям статьи буду снижать размер теста но при этом обучающая выборка остается гиперпараметром

In [3]:
import pandas as pd
HOUR_SECONDS = 60 * 60
DAY_SECONDS = 24 * HOUR_SECONDS

GAP_SIZE = HOUR_SECONDS // 2
TEST_SIZE = 1 * DAY_SECONDS

data = pd.read_parquet("multi_event.parquet", engine="fastparquet")
data = data[data['timestamp'] > 250 * DAY_SECONDS]

LAST_TIMESTAMP = data["timestamp"].max()

TEST_TIMESTAMP = LAST_TIMESTAMP - TEST_SIZE

TRAIN_TIMESTAMP = TEST_TIMESTAMP - GAP_SIZE


In [4]:
train_df = data[data['timestamp'] < TRAIN_TIMESTAMP]
test_df = data[data['timestamp'] > TEST_TIMESTAMP]

In [5]:
print(len(test_df))
print(len(train_df))

train_df, test_df , item_id_map = codic_of_index(train_df, test_df)

train_df_likes = train_df[~(train_df["event_type"] == "listen")]
test_df_likes = test_df[~(test_df["event_type"] == "listen")]

train_df_listen = train_df[train_df["event_type"] == "listen"]
test_df_listen = test_df[test_df["event_type"] == "listen"]


train_df_listen, stats = run_listen_pipeline(train_df_listen,
                                            ["clean_listen_duplicates", 
                                             "remove_not_valid_listen_data", 
                                             "cut_track_len", 
                                             "remove_rer_data"])


test_df_listen, stats = run_listen_pipeline(test_df_listen,
                                            ["clean_listen_duplicates", 
                                             "remove_not_valid_listen_data"])


train_df_likes, stats = run_listen_pipeline(train_df_likes,
                                            ["convert_reaction", 
                                             "rename_events"])


test_df_likes, stats = run_listen_pipeline(test_df_likes,
                                            ["convert_reaction", 
                                             "rename_events"])


test_df = pd.concat([test_df_likes,test_df_listen ]) 
train_df = pd.concat([train_df_likes,train_df_listen ]) 

print(len(test_df))
print(len(train_df))

238398
10510818
216480
9601733


## Models

In [6]:
from evaluate import evaluate_model
from models import *

/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Baseline


Самое базовое - взять топ 10 самых частых, при этом есть 2 варианта - по лайкам или по прослушиванию. В данном случае есть 2 реализации
Как бейзлайн возьмем топ 10 понравившихся песен всем пользователем. Возьмем именно по лайкам, так как они наиболее надежные чем прослушивания (однако для чистоты эксперимента были провериины и этот вариант, точность оказалась ниже)

In [8]:
model = MostPop_by_likes()
model.fit(train_df)

evaluate_model(model, train_df, test_df)

100%|███████████████████████████████████████████████████████████████████████████████████████| 4999/4999 [00:00<00:00, 15215.52it/s]

Mean Recall@10: 0.007387297749595295
Mean NDCG@10: 0.020734619374920698


In [9]:
model = MostPop_by_listen()
model.fit(train_df)

evaluate_model(model, train_df, test_df)

100%|███████████████████████████████████████████████████████████████████████████████████████| 4999/4999 [00:00<00:00, 12802.27it/s]

Mean Recall@10: 0.007430349887540012
Mean NDCG@10: 0.023781866335163746


In [ ]:
# ✅ 2. Построить популярность по сегментам

# Можно рассчитать:

# популярные треки

# популярные артисты

# популярные альбомы

# новинки, которые часто лайкают

# тренды по последним N дням

# Полезно как fallback.



# ✅ 3. Построить контентно-ориентированные рекомендации

# Из таблицы item–album–artist можно сделать:

# ✔ Рекомендации похожих исполнителей

# если пользователь лайкает Artist A → дать похожих артистов.

# ✔ Рекомендации по альбомам

# «Ты лайкал треки из альбома X — возможно тебе понравится Y».

## Collaborative filtering

### ALS

AlternatingLeastSquares (ALS) — это модель матричной факторизации для неявных откликов (implicit feedback).
Она раскладывает матрицу взаимодействий пользователей и объектов на две матрицы эмбеддингов — пользователей и айтемов — так, чтобы их скалярное произведение предсказывало интерес.
Обучается поочерёдным решением задач наименьших квадратов: сначала фиксирует айтемы и обновляет пользователей, потом наоборот.


В одной итерации ALS поочерёдно пересчитывает вектора пользователей и айтемов.
Сначала, при фиксированных айтемах, решаются задачи наименьших квадратов для всех пользователей;
потом, при фиксированных пользователях — для всех айтемов.
Это повторяется несколько итераций, пока модель не сойдётся.

Для обучения ALS требуется подготовить данные в формате матрицы взаимодействий, где каждая ячейка отражает степень интереса пользователя к объекту. Поскольку в нашем случае используются implicit-сигналы, нам необходимо преобразовать исходные события (прослушивание, лайк, дизлайк) в числовые веса.

На этапе подготовки данных мы подсчитываем, сколько раз пользователь прослушал каждый трек, а также приводим реакции к бинарному виду:
в результирующем датафрейме поле like получает значение 1,
аналогично dislike — значение 1.

В отдельном модуле реализована функция объединения данных, которая формирует итоговый датафрейм с колонками:

* признак лайка (like = 1)
* признак дизлайка (dislike = 1)
* количество прослушиваний каждой пары пользователь–трек

In [8]:
from cf_utils import *

train_df_als = merge_data_by_count(train_df)

# формула # Применим логорифмирование 
train_df_als["conf"] = (train_df_als["have_like"] * 20.0) + (train_df_als["have_dislike"] * -10) + np.log1p(train_df_als["listen_count"]*(train_df_als["played_ratio_pct"]/100))


Для ALS необходима разреженная матрица вида:

(user_id, item_id) → weight


где weight — это итоговая сила взаимодействия (прослушивания + реакции).
В модуле реализована функция, которая на основе сформированного датафрейма строит такую матрицу через coo_matrix и возвращает её в подходящем для модели формате.

In [264]:
matrix = create_user_item_matrix(train_df_als)
matrix

<Compressed Sparse Row sparse matrix of dtype 'int32'
	with 2943306 stored elements and shape (9954, 376570)>

In [265]:
model = ALS()
model.fit(matrix)

/opt/venv/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:21<00:00,  1.39it/s]


In [266]:
evaluate_model(model, train_df, test_df)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 4982/4982 [01:29<00:00, 55.96it/s]

Mean Recall@10: 0.04317543266682411
Mean NDCG@10: 0.1087657954592817


Метод ALS работает только с пользователями и объектами, которые присутствовали в обучающей выборке. Если в тесте встречается пользователь, которого модель ранее не видела, для него невозможно построить профиль в факторизационном пространстве.

В такой ситуации ALS не способен выдать рекомендации — в классе ALS такая логика приводит к возврату пустого списка.

Поэтому при использовании ALS важно учитывать, что:

* рекомендации гарантированно работают только для пользователей из train,
* все новые или ранее невстречавшиеся пользователи будут давать «пустой» результат.

Это естественное ограничение моделей матричной факторизации, основанных на implicit-feedback данных.

### ItemItemRec

In [42]:
model = ItemItemRec()
model.fit(matrix)

evaluate_model(model, train_df, test_df)

100%|████████████████████████████████████████████████████████████████████████████████| 4982/4982 [00:09<00:00, 498.59it/s]

Mean Recall@10: 0.0070190327978919
Mean NDCG@10: 0.017258051884734435


У авторов в статье точность была намного выше - возможно из-за того, что обучались на всем наборе данных, а не на одном месяце

### BPR

In [49]:
# BPR ожидает ITEM×USER матрицу, поэтому транспонируем
# (это стандарт для implicit: матрица item-user)

item_user_matrix = matrix.T.tocsr()

model = BPR()
model.fit(matrix)

evaluate_model(model, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4982/4982 [01:46<00:00, 46.99it/s]

Mean Recall@10: 0.022100891777865676
Mean NDCG@10: 0.05985546397300414


## Content_based 

Content-Based Filtering — это метод рекомендаций, который подбирает объекты, похожие на те, что пользователь уже оценил, используя их содержательные признаки (жанры, embedding-и, особенности контента). Его сильная сторона в том, что он не зависит от других пользователей, хорошо работает для новых объектов (нет cold-start для items) и даёт легко объяснимые рекомендации. Однако у подхода есть ограничения: качество сильно зависит от самих признаков, модель не умеет учитывать тренды или контекст поведения, а рекомендации могут быть слишком однообразными и «сужать» вкусы пользователя.


### by metadata

В наличии методанные, которые можно использовать как фичи - попробуем

In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import faiss
from tqdm import tqdm


In [10]:
items_meta = pd.read_parquet("items_meta.parquet")
items_meta["item_id"] = items_meta["item_id"].map(item_id_map)
items_meta = items_meta.drop_duplicates(subset=["item_id"])
items_meta = items_meta.dropna()

In [43]:
# items_meta['artist_id_freq'] = items_meta['artist_id'].map(items_meta['artist_id'].value_counts())
# items_meta['album_id_freq'] = items_meta['album_id'].map(items_meta['album_id'].value_counts())
# items_meta['track_length_seconds_freq'] = items_meta['track_length_seconds'].map(items_meta['track_length_seconds'].value_counts())
items_meta

,item_id,track_length_seconds,artist_id,album_id,artist_id_freq,album_id_freq,track_length_seconds_freq
0,16537.0,170.0,714356.0,1060653.0,29,2,10717
2,2905.0,105.0,1043457.0,547645.0,28,1,2592
3,84.0,185.0,94924.0,141751.0,113,17,13741
4,16.0,240.0,94924.0,1632076.0,113,8,11941
5,52084.0,130.0,997338.0,403561.0,503,12,5650
...,...,...,...,...,...,...,...
1334749,468758.0,245.0,416983.0,3354988.0,8,8,10578
1334750,468759.0,190.0,416983.0,3354988.0,8,8,14422
1334751,468761.0,275.0,6330.0,760322.0,1,1,6178
1334752,468764.0,170.0,1227844.0,318619.0,24,1,10717


Методы точного поиска ближайших соседей (kNN) требуют вычислять расстояние между вектором-запросом и каждым объектом в базе, поэтому их сложность линейная 
𝑂
(
𝑁
)
O(N), и на больших наборах данных с десятками или сотнями тысяч элементов они становятся крайне медленными: нужно выполнить миллионы скалярных произведений при каждом запросе. По этой причине в рекомендациях и поиске похожих объектов чаще используют ANN (Approximate Nearest Neighbors) — алгоритмы, которые строят специальный индекс (HNSW, Annoy, Faiss и др.) и ищут ближайших соседей не полностью перебирая весь набор, а умно обходя лишь небольшую часть точек. В результате скорость поиска ускоряется в десятки–сотни раз при минимальной потере качества, что делает ANN практически стандартом для больших эмбеддингов.

In [40]:
class CBF_by_metadata:
    def __init__(self):
        pass

    def fit(self, items_meta, data):
        self.data = data
        items_meta['artist_id'] = items_meta['artist_id'].map(items_meta['artist_id'].value_counts())
        items_meta['album_id'] = items_meta['album_id'].map(items_meta['album_id'].value_counts())
        items_meta['track_length_seconds'] = items_meta['track_length_seconds'].map(items_meta['track_length_seconds'].value_counts())
        
        X = items_meta[["track_length_seconds", "artist_id", "album_id"]]
        self.item_ids = items_meta["item_id"].to_numpy()
        

        X_batch = X.to_numpy().astype("float32")
        # X_batch = np.asarray(X_batch, dtype='float32')  # ensure numpy + float32
        self.X_batch = np.ascontiguousarray(X_batch)    # ensure C-contiguous
        print(self.X_batch)
        
        N, D = self.X_batch.shape
        self.index = faiss.IndexFlatIP(D)
        
        faiss.normalize_L2(self.X_batch)
        self.index.add(self.X_batch)
        
        print("Всего в индексе:", self.index.ntotal)
        
        self.id2pos = {iid: i for i, iid in enumerate(self.item_ids)} # Индексы реальные и в матрице - разные. Нужно для сопоставления

    # Профиль пользователя = средний вектор всех айтемов, которые он слушал.
    def build_user_profile_embed(self, uid):
        listened_items = self.data[self.data["uid"] == uid]["item_id"].values
    
        indices = [self.id2pos[iid] for iid in listened_items if iid in self.id2pos]
        if indices == []:
            return None
        
        # берём вектора всех прослушанных треков
        vectors = self.X_batch[indices]
        
        # пользовательский профиль = средний вектор
        # user_vec = vectors.ьу(axis=0)
        user_vec = np.median(vectors, axis=0)
        user_vec = np.asarray(user_vec)        # → ndarray (1, D)
    
        return np.array([user_vec])

    def similar_tracks(self, vec, k=10): #

        sims, ids = self.index.search(vec, k+1)
    
        sims = sims[0]
        ids = ids[0]
        
        # убираем элемент, где id == pos
        result = []
        for track_pos, score in zip(ids, sims):
            result.append((self.item_ids[track_pos], float(score)))
            if len(result) == k:
                break
    
        res = [pair[0] for pair in result[:k]]
        return res

    def recommend(self, uid, k = 10):
        vec = self.build_user_profile_embed(uid)
        if vec is None:
            return [], []
        # print(vec)
        rec = self.similar_tracks(vec)
        # print(rec)

        return rec, []



class CBF_by_embeding:
    def __init__(self):
        pass

    def fit(self, items_meta, data):
        self.data = data

        self.item_ids = items_meta["item_id"].to_numpy()
        self.X_batch = np.vstack(items_meta["normalized_embed"].to_list()).astype("float32")   # (N, D)
        
        N, D = self.X_batch.shape
        self.index = faiss.IndexFlatIP(D)
        
        self.index.add(self.X_batch)
        
        print("Всего в индексе:", self.index.ntotal)
        
        self.id2pos = {iid: i for i, iid in enumerate(self.item_ids)} # Индексы реальные и в матрице - разные. Нужно для сопоставления

    # Профиль пользователя = средний вектор всех айтемов, которые он слушал.
    def build_user_profile_embed(self, uid):
        listened_items = self.data[self.data["uid"] == uid]["item_id"].values
    
        indices = [self.id2pos[iid] for iid in listened_items if iid in self.id2pos]
        if indices == []:
            return None
        
        # берём вектора всех прослушанных треков
        vectors = self.X_batch[indices]
        
        # пользовательский профиль = средний вектор
        # user_vec = vectors.median(axis=0)
        user_vec = np.median(vectors, axis=0)
        user_vec = np.asarray(user_vec)        # → ndarray (1, D)
    
        return np.array([user_vec])

    def similar_tracks(self, vec, k=10): #

        sims, ids = self.index.search(vec, k+1)
    
        sims = sims[0]
        ids = ids[0]
        
        # убираем элемент, где id == pos
        result = []
        for track_pos, score in zip(ids, sims):
            result.append((self.item_ids[track_pos], float(score)))
            if len(result) == k:
                break

        return [pair[0] for pair in result[:k]], [pair[1] for pair in result[:k]] 

    def recommend(self, uid, k = 10):
        vec = self.build_user_profile_embed(uid)
        if vec is None:
            return [], []
        # print(vec)

    
        return self.similar_tracks(vec)



In [53]:
model = CBF_by_metadata()
content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>3) | (train_df_als["have_like"]==1)]

model.fit(items_meta, content_based_train)





/tmp/ipykernel_42743/2073778428.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>3) | (train_df_als["have_like"]==1)]


[[1.0717e+04 2.9000e+01 2.0000e+00]
 [2.5920e+03 2.8000e+01 1.0000e+00]
 [1.3741e+04 1.1300e+02 1.7000e+01]
 ...
 [6.1780e+03 1.0000e+00 1.0000e+00]
 [1.0717e+04 2.4000e+01 1.0000e+00]
 [6.7110e+03 6.1000e+01 1.0000e+00]]
Всего в индексе: 450544


In [45]:
evaluate_model(model, content_based_train, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 4999/4999 [00:10<00:00, 473.31it/s]

Mean Recall@10: 0.00025586438303550014
Mean NDCG@10: 0.0007824207787830228


In [24]:
import polars as pl
import numpy as np
# import umap.umap_ as umap
import matplotlib.pyplot as plt

with open("item_id_mapping.json", "r") as f:
    map_bad = json.load(f)

lf = pl.scan_parquet("filtered_embeddings.parquet")

embed_df = (
    lf
    .slice(50_000, 50_000)      # offset = 50k, length = 50k
    .select(["item_id", "normalized_embed"])
    .collect()
)


embed_df = embed_df.to_pandas()
# embed_df["item_id"] = embed_df["item_id"].map(map_bad)
embed_df["item_id"] = embed_df["item_id"].map(item_id_map)
embed_df = embed_df.dropna()

In [25]:
embed_df["item_id"]


0        192737.0
1        318829.0
2        331247.0
4        361282.0
6         18107.0
           ...   
49993    300677.0
49995    322052.0
49996    295458.0
49997    335690.0
49999    160279.0
Name: item_id, Length: 25932, dtype: float64

In [33]:
map_bad['318829']

318829

In [38]:
import faiss


model = CBF_by_embeding()
content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>10) | (train_df_als["have_like"]==1)]

model.fit(embed_df, content_based_train)

/tmp/ipykernel_42743/114747141.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>10) | (train_df_als["have_like"]==1)]


Всего в индексе: 25932


In [15]:
content_based_train

,uid,item_id,listen_count,played_ratio_pct,have_dislike,have_like,conf
37,0,38,4.0,100.00,0.0,1.0,21.609438
122,0,124,4.0,100.00,0.0,1.0,21.609438
127,0,129,5.0,100.00,0.0,1.0,21.791759
152,0,154,4.0,75.25,0.0,1.0,21.388791
179,0,181,5.0,100.00,0.0,1.0,21.791759
...,...,...,...,...,...,...,...
4253744,9960,18188,0.0,0.00,0.0,1.0,20.000000
4253745,9960,26523,0.0,0.00,0.0,1.0,20.000000
4253746,9960,63704,0.0,0.00,0.0,1.0,20.000000
4253747,9966,313,0.0,0.00,0.0,1.0,20.000000


In [39]:

from tqdm import tqdm
from metrics import ndcg_at_k, recall_at_k
import numpy as np

def evaluate_model(model, train_df, test_df, k=10):
    grouped_users = test_df.groupby("uid")
    overall_recall = []
    overall_ndcg = []
    for uid, group in tqdm(grouped_users):
        # print(list(group["item_id"]))
        user_true = list(set(group["item_id"]))
        
        rec, w = model.recommend(uid, k)
        
        recall = recall_at_k(rec, user_true,  10)
        ndcg = ndcg_at_k(rec ,user_true,  10)


        # if ndcg>0:
        #     print("---------")
        #     print(ndcg, uid, user_true, rec, w)
        
        overall_recall.append(recall)
        overall_ndcg.append(ndcg)
    
    print("Mean Recall@10:", np.mean(overall_recall))
    print("Mean NDCG@10:",np.mean(overall_ndcg))


evaluate_model(model, content_based_train, test_df)

100%|████████████████████████████████████████████████████████████████████████████████████████| 4999/4999 [00:04<00:00, 1097.60it/s]


Mean Recall@10: 0.005702681249350968
Mean NDCG@10: 0.022336459933116434


In [61]:
content_based_train# ["item_id"]

1              1
2              2
11            11
15            16
16            17
           ...  
4253744    18188
4253745    26523
4253746    63704
4253747      313
4253748    40060
Name: item_id, Length: 646000, dtype: int64

In [60]:
items_meta

,item_id,track_length_seconds,artist_id,album_id
0,16537.0,10717,29,2
2,2905.0,2592,28,1
3,84.0,13741,113,17
4,16.0,11941,113,8
5,52084.0,5650,503,12
...,...,...,...,...
1334749,468758.0,10578,8,8
1334750,468759.0,14422,8,8
1334751,468761.0,6178,1,1
1334752,468764.0,10717,24,1


In [11]:
df_merged = train_df.merge(items_meta[['item_id', 'artist_id', 'album_id']], 
                      on='item_id', 
                      how='left')

In [12]:
df_merged

,uid,timestamp,item_id,is_organic,played_ratio_pct,track_length_seconds,event_type,artist_id,album_id
0,0,23471875,124,0.0,NaN,NaN,like,317787.0,367130.0
1,0,23540595,129,0.0,NaN,NaN,like,258758.0,3111638.0
2,0,23865945,154,1.0,NaN,NaN,like,126968.0,19518.0
3,0,24148190,181,0.0,NaN,NaN,like,19909.0,1820057.0
4,0,24642765,38,1.0,NaN,NaN,like,539376.0,1722281.0
...,...,...,...,...,...,...,...,...,...
9601728,9967,21803155,35280,0.0,100.0,165.0,listen,406588.0,598856.0
9601729,9967,22065175,35280,0.0,100.0,165.0,listen,406588.0,598856.0
9601730,9967,25264920,38768,1.0,99.0,160.0,listen,934172.0,1296847.0
9601731,9967,25262450,44295,1.0,100.0,225.0,listen,1087272.0,704310.0


In [8]:
train_df #[df_merged["uid"] == 

,uid,timestamp,item_id,is_organic,played_ratio_pct,track_length_seconds,event_type
2922,0,23471875,124,0.0,NaN,NaN,like
2937,0,23540595,129,0.0,NaN,NaN,like
2963,0,23865945,154,1.0,NaN,NaN,like
3004,0,24148190,181,0.0,NaN,NaN,like
3162,0,24642765,38,1.0,NaN,NaN,like
...,...,...,...,...,...,...,...
10080168,9967,21803155,35280,0.0,100.0,165.0,listen
10080169,9967,22065175,35280,0.0,100.0,165.0,listen
10080170,9967,25264920,38768,1.0,99.0,160.0,listen
10080171,9967,25262450,44295,1.0,100.0,225.0,listen


In [75]:
df_merged[(df_merged["event_type"] == "like") & (df_merged["uid"] == 0)]

,uid,timestamp,item_id,is_organic,played_ratio_pct,track_length_seconds,event_type,artist_id,album_id
0,0,23471875,124,0.0,NaN,NaN,like,317787.0,367130.0
1,0,23540595,129,0.0,NaN,NaN,like,258758.0,3111638.0
2,0,23865945,154,1.0,NaN,NaN,like,126968.0,19518.0
3,0,24148190,181,0.0,NaN,NaN,like,19909.0,1820057.0
4,0,24642765,38,1.0,NaN,NaN,like,539376.0,1722281.0
5,0,25249855,191,1.0,NaN,NaN,like,1283582.0,3166650.0


,artist_id,item_id,track_likes,artist_likes,artist_rank,track_rank
0,19909.0,181,1,1,1,1
1,126968.0,154,1,1,2,1
2,258758.0,129,1,1,3,1
3,317787.0,124,1,1,4,1
4,539376.0,38,1,1,5,1
5,1283582.0,191,1,1,6,1


In [16]:
class MostPop_by_artist:
    def __init__(self):
        pass

    def fit(self, df_merged):
         # 1. Берём только лайки
        likes = df_merged[df_merged["event_type"] == "like"].copy()

        listen = (
                df_merged[df_merged["event_type"] == "listen"]
                .groupby(['uid', 'item_id'])
                .agg({
                    'timestamp': 'count',
                    'artist_id': 'first',
                    
                })
                .reset_index()
            )
        
        listen = listen[listen["timestamp"]>5]

        likes = pd.concat([likes, listen])

        self.likes = likes

        
    def ger_rec(self, uid, likes):
        N_ARTISTS = 10   # сколько самых популярных артистов
        N_TRACKS = 10    # сколько треков на каждого артиста

        likes = likes[likes["uid"] == uid]
        # 2. Считаем, сколько лайков у каждого артиста
        artist_like_counts = (
            likes.groupby("artist_id")["item_id"]
            .count()
            .sort_values(ascending=False)
            .reset_index(name="artist_likes")
        )
        
        # 3. Берём топ-10 артистов
        top_artists = artist_like_counts.head(N_ARTISTS).copy()
        top_artists["artist_rank"] = range(1, len(top_artists) + 1)
        
        # 4. Оставляем лайки только по этим артистам
        likes_top = likes[likes["artist_id"].isin(top_artists["artist_id"])]
        
        # 5. Считаем лайки по (artist_id, item_id) — популярность треков у артиста
        track_counts = (
            likes_top.groupby(["artist_id", "item_id"])
            .size()
            .reset_index(name="track_likes")
        )
        
        # 6. Подтягиваем к трекам общие лайки артиста и его ранг
        track_counts = track_counts.merge(top_artists, on="artist_id", how="left")
        
        # 7. Сортируем: сначала по популярности артиста, потом по популярности трека
        track_counts = track_counts.sort_values(
            ["artist_rank", "track_likes"],
            ascending=[True, False]
        )
        
        # 8. Берём топ N_TRACKS треков для каждого артиста и добавляем ранг трека
        top_tracks_per_artist = (
            track_counts
            .groupby("artist_id")
            .head(N_TRACKS)
            .copy()
        )
        
        top_tracks_per_artist["track_rank"] = (
            top_tracks_per_artist.groupby("artist_id")["track_likes"]
            .rank(method="first", ascending=False)
            .astype(int)
        )
        
        # 9. Финальная сортировка для красивого вывода
        top_tracks_per_artist = top_tracks_per_artist.sort_values(
            ["artist_rank", "track_rank"]
        ).reset_index(drop=True)
        
        # self.rec = top_tracks_per_artist.sort_values("track_likes", ascending = False)[:10]["item_id"].tolist()
        # self.weights = []

        return top_tracks_per_artist.sort_values("track_likes", ascending = False)[:10]["item_id"].tolist(), []

    def recommend(self, uid, k = 10):
        rec, weights = self.ger_rec(uid, self.likes)
        return rec[:k], weights[:k]
        

In [17]:
model = MostPop_by_artist()
model.fit(df_merged)

evaluate_model(model, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 4999/4999 [00:27<00:00, 181.54it/s]

Mean Recall@10: 0.06902875818118087
Mean NDCG@10: 0.15650630564423726


In [59]:
class MostPop_by_personal_artist:
    def __init__(self):
        pass

    def fit(self, df):
        stat = df[df["event_type"] == "like"]["item_id"].value_counts()
        self.rec = stat.index.tolist()
        self.weights = stat.tolist()
        print(self.rec[:10])
        

    def recommend(self, uid, k = 10):
        return self.rec[:k], self.weights[:k]

train_df_als = merge_data_by_count(train_df)

# формула # Применим логорифмирование 
model = MostPop_by_likes()
model.fit(df_merged)

[4453, 664, 8272, 486, 2797, 307, 2776, 124, 4599, 2898]


In [70]:
df_merged

,uid,timestamp,item_id,is_organic,played_ratio_pct,track_length_seconds,event_type,artist_id,album_id
0,0,23471875,124,0.0,NaN,NaN,like,317787.0,367130.0
1,0,23540595,129,0.0,NaN,NaN,like,258758.0,3111638.0
2,0,23865945,154,1.0,NaN,NaN,like,126968.0,19518.0
3,0,24148190,181,0.0,NaN,NaN,like,19909.0,1820057.0
4,0,24642765,38,1.0,NaN,NaN,like,539376.0,1722281.0
...,...,...,...,...,...,...,...,...,...
9601728,9967,21803155,35280,0.0,100.0,165.0,listen,406588.0,598856.0
9601729,9967,22065175,35280,0.0,100.0,165.0,listen,406588.0,598856.0
9601730,9967,25264920,38768,1.0,99.0,160.0,listen,934172.0,1296847.0
9601731,9967,25262450,44295,1.0,100.0,225.0,listen,1087272.0,704310.0
